### Mounting drive on Colab

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


###Libraries

In [2]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [3]:
import os
import cv2
import os
import h5py
import numpy as np
import pickle
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
import tensorflow_addons as tfa
os.chdir('/gdrive/My Drive/')
from numpy import savez_compressed
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import tensorflow_datasets as tfds
from tensorflow_addons.metrics import HammingLoss
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

## Effect of changing Epochs

### For ImageNet as a source

In [5]:
from tensorflow.keras.applications.resnet50 import ResNet50

from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()

pre_trained_model = ResNet50(input_shape = (96, 96, 3), 
                                include_top = False, 
                                weights = None)
#for layer in pre_trained_model.layers:
#  layer.trainable = False

In [6]:
def compile_model():
  from tensorflow.keras.optimizers import RMSprop

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(pre_trained_model.output)

  # Add a fully connected layer with "num_neurons" hidden units and ReLU activation
  x = layers.Dense(2000, activation='relu')(x)

  x = layers.Dense(1000, activation='relu')(x)

  x = layers.Dense(500, activation='relu')(x)                  
  
  # Add a final sigmoid layer for classification
  x = layers.Dense(15, activation='softmax')(x)           

  model = Model( pre_trained_model.input, x) 

  model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc', tf.keras.metrics.AUC(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), HammingLoss(threshold=0.5, mode='multiclass'), F1Score(num_classes= 15, threshold=0.5)])
  return model

In [7]:
data = np.load('Xtrain.npy', allow_pickle=True)
labels = np.load('ytrain.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42, stratify = labels)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42, stratify = y_test)

In [8]:
y_train = to_categorical(y_train, 15)
y_val = to_categorical(y_val, 15)
y_test = to_categorical(y_test, 15)

In [9]:
for i in [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]:
  print("*********** Fitting model for", i ,"epochs ***************")
  model = compile_model()
  model.fit(X_train, y_train, batch_size = 120, epochs= i, validation_data=(X_val, y_val), callbacks = [cb])
  test_acc = model.evaluate(X_test, y_test)
  print("***********test accuracy is", test_acc,"***************")
  print("***********train time is", sum(cb.logs),"***************")
  print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 2 epochs ***************


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/2
200/200 [==============================] - 156s 641ms/step - loss: 3.1693 - acc: 0.0777 - auc: 0.5263 - recall: 0.0043 - precision: 0.0900 - hamming_loss: 0.9957 - f1_score: 0.0080 - val_loss: 2.7474 - val_acc: 0.0667 - val_auc: 0.5104 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/2
94/94 [==============================] - 9s 71ms/step - loss: 2.5566 - acc: 0.1847 - auc: 0.7085 - recall: 0.0173 - precision: 0.6265 - hamming_loss: 0.9827 - f1_score: 0.0303
***********test accuracy is [2.556617259979248, 0.18466666340827942, 0.7084688544273376, 0.01733333244919777, 0.6265060305595398, 0.9826666712760925, array([0.        , 0.        , 0.        , 0.00995025, 0.        ,
       0.23045269, 0.01980198, 0.        , 0.00952381, 0.        ,
       0.0097561 , 0.01941748, 0.1559633 , 0.        , 0.        ],
      dtype=float32)] ***************
***********train time is 283.287819069 ***************
f1 score is 0.03